<a href="https://colab.research.google.com/github/janishjindal/Google-Colab-Notebooks/blob/main/AffableProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving affable_data_orig.csv to affable_data_orig.csv


In [ ]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['data.csv']))

print(df)

               username  ...  hasEmail
0           helthematic  ...     False
1              shipliam  ...      True
2         thesisurinary  ...      True
3      ashesphentermine  ...      True
4             helwillow  ...     False
..                  ...  ...       ...
994      magicphenotype  ...      True
995          stiffmagic  ...      True
996      diggsdelivered  ...     False
997       truthmatching  ...     False
998  infectiousstanding  ...      True

[999 rows x 21 columns]


Data Engineered feature extraction from Image Label Tags

In [ ]:
## list of person-like features extracted from imageLabels column using data-engineering
personFeatures = {'neck', 'hair', 'eyelash', 'beard', 'human leg', 'face', 'facial expression', 'lip', 'chin', 'moustache', 'blond', 'human body', 'arm', 'human', 'waist', 'man', 'woman', 'wrist', 'finger', 'nose', 'person', 'leg', 'cheek', 'tongue', 'lady', 'muscle', 'forehead', 'shoulder', 'eye', 'boy', 'mouth', 'male', 'girl', 'skin', 'thigh', 'smile', 'chest', 'nail','head', 'hand', 'people', 'eyebrow', 'layered hair', 'hair coloring', 'brown hair', 'facial hair', 'hair accessory', 'thumb', 'jaw', 'hip'}

Feature expansion from Col ImageLabels (creating cols: newImageLabels and isPerson)

In [ ]:
## Creating a new column (newImageLabels) which contains imageLabel tags above a particular confidence threshold 
## Creating a new feature (isPerson) depicting if an imageLabel tag contains person-like features above a particular confidence threshold

import ast
total_rows = len(df)
isPerson = []
confidence_threshold = 0.65
facialFeature_condifence = 0.0
newDfColumn = []
for row in range(total_rows):
  try:
    imageLabelList = ast.literal_eval(df.imageLabels[row])
    newImageLabelList = []
    for tup in imageLabelList :
      if(tup[1] >= confidence_threshold):
        newImageLabelList.append(tup)
    pf_count = 0
    for tup in newImageLabelList:
      if(tup[0] in personFeatures):
        pf_count+=1
    pf_percent = pf_count/len(newImageLabelList)
    newDfColumn.append(newImageLabelList)
    if(pf_percent > facialFeature_condifence):
      isPerson.append(1)
    else:
      isPerson.append(0)
  except ValueError:
    #print("Encountered null")
    newDfColumn.append([])
    isPerson.append(0)
    continue
  except ZeroDivisionError:
    # newImageList is empty => no image label with high confidence tag
    newDfColumn.append([])
    isPerson.append(0)
    continue
  except:
    newDfColumn.append([])
    isPerson.append(0)
    continue
df["newImageLabels"] = newDfColumn
df["isPerson"] = isPerson

Bag of Word (BoW) on ImageLabel tags

In [ ]:
imageLabelBoWCol = []
for row in range(total_rows):
  try:
    imageLabelList = ast.literal_eval(df.imageLabels[row])
    bow = ""
    for tup in imageLabelList :
      if(tup[1] > confidence_threshold):
        bow += tup[0] + " " 
    imageLabelBoWCol.append(bow)    
  except ValueError:
    #print("Encountered null")
    imageLabelBoWCol.append("")
    continue
df["imageLabelBoW"] = imageLabelBoWCol

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   username             999 non-null    object 
 1   brand                999 non-null    bool   
 2   biography            946 non-null    object 
 3   mediaCount           993 non-null    float64
 4   cityId               641 non-null    float64
 5   latitude             637 non-null    float64
 6   longitude            642 non-null    float64
 7   zip                  203 non-null    object 
 8   contactPhoneNumber   414 non-null    float64
 9   followingCount       993 non-null    float64
 10  followerCount        993 non-null    float64
 11  usertagsCount        993 non-null    float64
 12  isBusiness           806 non-null    object 
 13  externalUrl          584 non-null    object 
 14  category             642 non-null    object 
 15  geoMediaCount        993 non-null    flo

In [ ]:
df.head(10)

In [ ]:
df_orig=df.copy()

Feature Selection : Removing unwanted cols based on domain knowledge

In [ ]:
df.drop(['biography', 'username','externalUrl', 'picture', 'cityId', 'latitude', 'longitude', 'zip', 'contactPhoneNumber', 'imageLabels', 'newImageLabels'], axis = 1, inplace=True)

Applying CountVectorizer/Tfidf Vectorizer on ImageLabelBoW

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfVectorizer
cv=CountVectorizer()
#cv=TfidfVectorizer()
cv.fit(df['imageLabelBoW'])
cv_fit = cv.transform(df['imageLabelBoW'])
bow_df = pd.DataFrame(cv_fit.toarray(),columns=cv.get_feature_names_out())
bow_df.rename(columns = {'brand':'brand_bow'}, inplace = True)
bow_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Columns: 717 entries, abdomen to youth
dtypes: int64(717)
memory usage: 5.5 MB


Handling Missing Values

In [ ]:
numerical_vars = ['mediaCount', 'followingCount', 'followerCount', 'usertagsCount', 'geoMediaCount', 'shoppablePostsCount', 'followingTagCount']
for i in numerical_vars:
  df[i].fillna(df[i].mean(), inplace=True)

categorical_vars = ['isBusiness', 'category', 'hasEmail', 'isPerson']
for i in categorical_vars:
  df[i].fillna(df[i].mode()[0], inplace=True)

Concatenating CountVectorizer of ImageLabels with Original df 

In [ ]:
df = pd.concat([df,bow_df], axis=1)
df.drop('imageLabelBoW', axis=1, inplace=True)

One-hot encoding for categical vars

In [ ]:
from sklearn.preprocessing import OneHotEncoder
categorical_vars = ['isBusiness', 'category', 'hasEmail', 'isPerson']

category_encoder = OneHotEncoder(handle_unknown="ignore")
category_encoder.fit(df[['category']])
category_array = category_encoder.transform(df[['category']]).toarray()
category_ohe_df = pd.DataFrame(category_array, columns=category_encoder.get_feature_names_out(['category']))

isBusiness_encoder = OneHotEncoder(handle_unknown="ignore")
isBusiness_encoder.fit(df[['isBusiness']])
isBusiness_array = isBusiness_encoder.transform(df[['isBusiness']]).toarray()
isBusiness_ohe_df = pd.DataFrame(isBusiness_array, columns=isBusiness_encoder.get_feature_names_out(['isBusiness']))


hasEmail_encoder = OneHotEncoder(handle_unknown="ignore")
hasEmail_encoder.fit(df[['hasEmail']])
hasEmail_array = hasEmail_encoder.transform(df[['hasEmail']]).toarray()
hasEmail_ohe_df = pd.DataFrame(hasEmail_array, columns=hasEmail_encoder.get_feature_names_out(['hasEmail']))

isPerson_encoder = OneHotEncoder(handle_unknown="ignore")
isPerson_encoder.fit(df[['isPerson']])
isPerson_array = isPerson_encoder.transform(df[['isPerson']]).toarray()
isPerson_ohe_df = pd.DataFrame(isPerson_array, columns=isPerson_encoder.get_feature_names_out(['isPerson']))

ohe_df = pd.concat([category_ohe_df, isBusiness_ohe_df, hasEmail_ohe_df, isPerson_ohe_df], axis=1)
df = pd.concat([df, ohe_df], axis=1)
df.drop(categorical_vars, axis=1, inplace=True)

In [ ]:
df.tail(10)

,brand,mediaCount,followingCount,followerCount,usertagsCount,geoMediaCount,shoppablePostsCount,followingTagCount,abdomen,academic,accessories,accessory,acrylic,active,adventure,advertising,aerospace,air,aircraft,airplane,alaskan,album,alcohol,alcoholic,american,amphibian,and,angle,animal,annual,appetizer,aqua,area,arecales,arm,army,arranging,art,artist,arts,...,category_Publisher,category_Radio Station,category_Recreation & Sports Website,category_Region,category_Restaurant,category_Retail Company,category_School,"category_Science, Technology & Engineering",category_Scientist,category_Seafood Restaurant,category_Shopping & Retail,category_Social Media Agency,category_Society & Culture Website,category_Software,category_Sports,category_Sports & Recreation,category_Sports Event,category_Sports League,category_Sports Team,"category_Stadium, Arena & Sports Venue",category_TV,category_TV Channel,category_Tour Agency,category_Tourist Information Center,category_Vegetarian/Vegan Restaurant,category_Video Creator,category_Visual Arts,category_Vitamins/Supplements,category_Website,category_Wedding Planning Service,category_Wine/Spirits,category_Women's Clothing Store,category_Women's Health Clinic,category_Writer,isBusiness_False,isBusiness_True,hasEmail_False,hasEmail_True,isPerson_0,isPerson_1
989,False,882.0,701.0,1048.0,255.0,0.0,0.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
990,False,380.0,1199.0,12792.0,555.0,0.0,0.0,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
991,True,257.0,665.0,1025.0,53.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
992,True,1561.0,718.0,66433.0,7119.0,0.0,69.0,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
993,False,186.0,6278.0,103245.0,65.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
994,False,1002.0,909.0,28739.0,645.0,0.0,0.0,1.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
995,True,11.0,7035.0,23395.0,2.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
996,True,74.0,1.0,2118.0,442.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
997,False,6.0,648.0,445.0,32.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
998,True,147.0,132.0,4141.0,62.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

Dimensionality reduction using SVD 

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
tmp_df_BoW = df.copy()
tmp_df_BoW.drop(['brand'], axis=1, inplace=True)

# Scale data before applying SVD
scaling = StandardScaler() 

# Use fit and transform method
scaling.fit(tmp_df_BoW)
Scaled_data=scaling.transform(tmp_df_BoW)

svd=TruncatedSVD(n_components=20)
transformed_cv = svd.fit_transform(Scaled_data)
print('Transformed CV Shape: ',transformed_cv.shape)
SVD_df = pd.DataFrame(transformed_cv)

Transformed CV Shape:  (999, 20)


Training ML Models (Scaling + SVD data) and testing accuracies

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

x_train,x_test,y_train,y_test=train_test_split(SVD_df, df['brand'], test_size=.2, stratify=df['brand'] , random_state=3)

models=[
SVC(C=10,gamma=0.1,random_state=201,probability=True),
LogisticRegression(random_state=201),
RandomForestClassifier(n_estimators=200,random_state=201, criterion='entropy'),
AdaBoostClassifier(n_estimators=200,learning_rate=1,random_state=201),
XGBClassifier(n_estimators=200,learning_rate=1,random_state=201),
]

for model in models:
    model.fit(x_train, y_train)
    
    predictions = model.predict(x_test)
    
    probs = model.predict_proba(x_test)
    
    print('*******************************************','\n')
    print ('Model : ', model, '\n')

    print('Confusion Matrix :','\n',confusion_matrix(y_test,predictions))

    print('\n','\n','Classification Report :','\n',classification_report(y_test,predictions))
    
    print('\n','\n','Accuracy :',accuracy_score(y_test,predictions))

    #print(roc_auc_score(y_test, probs))

******************************************* 

Model :  SVC(C=10, gamma=0.1, probability=True, random_state=201) 

Confusion Matrix : 
 [[76 15]
 [19 90]]

 
 Classification Report : 
               precision    recall  f1-score   support

       False       0.80      0.84      0.82        91
        True       0.86      0.83      0.84       109

    accuracy                           0.83       200
   macro avg       0.83      0.83      0.83       200
weighted avg       0.83      0.83      0.83       200


 
 Accuracy : 0.83
******************************************* 

Model :  LogisticRegression(random_state=201) 

Confusion Matrix : 
 [[69 22]
 [14 95]]

 
 Classification Report : 
               precision    recall  f1-score   support

       False       0.83      0.76      0.79        91
        True       0.81      0.87      0.84       109

    accuracy                           0.82       200
   macro avg       0.82      0.81      0.82       200
weighted avg       0.82      0.82

Training ML Models (Original + No Scaling + No SVD) -> Best

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, accuracy_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

df_train = df[df.columns.drop(['brand'])]
x_train,x_test,y_train,y_test=train_test_split(df_train, df['brand'], test_size=.25, stratify=df['brand'] , random_state=3)

models=[
MultinomialNB(),
SVC(C=10,gamma=0.1,random_state=201,probability=True),
LogisticRegression(random_state=201),
RandomForestClassifier(n_estimators=200,random_state=201, criterion='entropy'),
AdaBoostClassifier(n_estimators=200,learning_rate=.3,random_state=201),
XGBClassifier(n_estimators=200,learning_rate=.2,random_state=3),
]

for model in models:
    model.fit(x_train, y_train)
    
    predictions = model.predict(x_test)

    df_debug = pd.concat([y_test, pd.DataFrame(predictions), x_test], axis=1)

    probs = model.predict_proba(x_test)
    
    print('*******************************************','\n')
    print ('Model : ', model, '\n')

    print('Confusion Matrix :','\n',confusion_matrix(y_test,predictions))

    print('\n','\n','Classification Report :','\n',classification_report(y_test,predictions))
    
    print('\n','\n','Accuracy :',accuracy_score(y_test,predictions))

    #print(roc_auc_score(y_test, probs, average='macro', multi_class='ovr'))

******************************************* 

Model :  MultinomialNB() 

Confusion Matrix : 
 [[ 26  88]
 [ 12 124]]

 
 Classification Report : 
               precision    recall  f1-score   support

       False       0.68      0.23      0.34       114
        True       0.58      0.91      0.71       136

    accuracy                           0.60       250
   macro avg       0.63      0.57      0.53       250
weighted avg       0.63      0.60      0.54       250


 
 Accuracy : 0.6
******************************************* 

Model :  SVC(C=10, gamma=0.1, probability=True, random_state=201) 

Confusion Matrix : 
 [[  3 111]
 [  0 136]]

 
 Classification Report : 
               precision    recall  f1-score   support

       False       1.00      0.03      0.05       114
        True       0.55      1.00      0.71       136

    accuracy                           0.56       250
   macro avg       0.78      0.51      0.38       250
weighted avg       0.76      0.56      0.41    

Training ML Models (Original + Scaling + No SVD)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, accuracy_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

x_train,x_test,y_train,y_test=train_test_split(Scaled_data, df['brand'], test_size=.2, stratify=df['brand'] , random_state=3)

models=[
SVC(C=10,gamma=.0001,random_state=201,probability=True),
LogisticRegression(random_state=201),
RandomForestClassifier(n_estimators=200,random_state=201),
AdaBoostClassifier(n_estimators=200,learning_rate=.3,random_state=201),
XGBClassifier(n_estimators=200,learning_rate=.2,random_state=201),
]

for model in models:
    model.fit(x_train, y_train)
    
    predictions = model.predict(x_test)
    
    probs = model.predict_proba(x_test)
    
    print('*******************************************','\n')
    print ('Model : ', model, '\n')

    print('Confusion Matrix :','\n',confusion_matrix(y_test,predictions))

    print('\n','\n','Classification Report :','\n',classification_report(y_test,predictions))
    
    print('\n','\n','Accuracy :',accuracy_score(y_test,predictions))

    #print(roc_auc_score(y_test, probs, average='macro', multi_class='ovr'))

******************************************* 

Model :  SVC(C=10, gamma=0.0001, probability=True, random_state=201) 

Confusion Matrix : 
 [[70 21]
 [21 88]]

 
 Classification Report : 
               precision    recall  f1-score   support

       False       0.77      0.77      0.77        91
        True       0.81      0.81      0.81       109

    accuracy                           0.79       200
   macro avg       0.79      0.79      0.79       200
weighted avg       0.79      0.79      0.79       200


 
 Accuracy : 0.79
******************************************* 

Model :  LogisticRegression(random_state=201) 

Confusion Matrix : 
 [[78 13]
 [31 78]]

 
 Classification Report : 
               precision    recall  f1-score   support

       False       0.72      0.86      0.78        91
        True       0.86      0.72      0.78       109

    accuracy                           0.78       200
   macro avg       0.79      0.79      0.78       200
weighted avg       0.79      0

Feature Tuning (GridSearch) for SVC (Scaling + SVD)

In [ ]:

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

dict1={'C':[.0001,0.001,.01,.1,1,10,100,1000],'gamma':[.00001,.0001,.001,.01,.1,1,10,100], 'kernel': ['rbf']}
#dict1={'C':[.001,.01,.1,1,10],'gamma':[.001,.01,.1,1,10], 'kernel': ['poly']}

model=SVC()

grid=GridSearchCV(estimator=model,param_grid=dict1,verbose=3,n_jobs=1,cv=2,)

df_train = df[df.columns.drop(['brand'])]
x_train,x_test,y_train,y_test=train_test_split(Scaled_data, df['brand'], test_size=.2, stratify=df['brand'] , random_state=3)

grid.fit(x_train,y_train)

print('Best Parameters : ',grid.best_params_,'\n')

print('Best Score : ',grid.best_score_,'\n')


Fitting 2 folds for each of 64 candidates, totalling 128 fits
[CV 1/2] END .C=0.0001, gamma=1e-05, kernel=rbf;, score=0.545 total time=   0.1s
[CV 2/2] END .C=0.0001, gamma=1e-05, kernel=rbf;, score=0.546 total time=   0.2s
[CV 1/2] END C=0.0001, gamma=0.0001, kernel=rbf;, score=0.545 total time=   0.2s
[CV 2/2] END C=0.0001, gamma=0.0001, kernel=rbf;, score=0.546 total time=   0.2s
[CV 1/2] END .C=0.0001, gamma=0.001, kernel=rbf;, score=0.545 total time=   0.2s
[CV 2/2] END .C=0.0001, gamma=0.001, kernel=rbf;, score=0.546 total time=   0.2s
[CV 1/2] END ..C=0.0001, gamma=0.01, kernel=rbf;, score=0.545 total time=   0.2s
[CV 2/2] END ..C=0.0001, gamma=0.01, kernel=rbf;, score=0.546 total time=   0.2s
[CV 1/2] END ...C=0.0001, gamma=0.1, kernel=rbf;, score=0.545 total time=   0.2s
[CV 2/2] END ...C=0.0001, gamma=0.1, kernel=rbf;, score=0.546 total time=   0.2s
[CV 1/2] END .....C=0.0001, gamma=1, kernel=rbf;, score=0.545 total time=   0.2s
[CV 2/2] END .....C=0.0001, gamma=1, kernel=rbf